Consistency is the key - and if snapchat taught us anything: it's that maintaining your streak is really motivating. This noteboo kaims to calculate longest streak of adhering to tracking your nutrition.

Luckily [Josh Devlin](https://joshdevlin.com/blog/calculate-streaks-in-pandas/) wrote a great article that will get us most of the way there!

In [2]:
import pandas as pd

data = pd.read_csv("./data.csv", index_col=0)

In [6]:
tracked_dates = pd.to_datetime(data['date'].unique())
tracked_dates = pd.DataFrame(tracked_dates)

In [7]:
tracked_dates['tracked']=True

In [14]:
min_date = tracked_dates[0].min()
max_date = tracked_dates[0].max()

In [15]:
import datetime as dt

# add some mock dates to test out streak calculations
min_date = min_date - dt.timedelta(days=3)
max_date = max_date + dt.timedelta(days=3)
all_dates = pd.Series(pd.date_range(min_date,max_date), name='all_dates')

streak_df = tracked_dates.merge(
    all_dates, how="outer", right_on="all_dates", left_on=0
    ).fillna(False)

From Josh's article

In [16]:
streak_df = streak_df.set_index("all_dates", drop=True).drop(0, axis=1)
streak_df = streak_df.sort_index()
streak_df['start_of_streak'] = streak_df['tracked'].ne(streak_df['tracked'].shift())
streak_df['streak_id'] = streak_df['start_of_streak'].cumsum()
streak_df['streak_counter'] = streak_df.groupby('streak_id').cumcount() + 1

streak_df

,tracked,start_of_streak,streak_id,streak_counter
all_dates,,,,
2022-11-24,False,True,1,1
2022-11-25,False,False,1,2
2022-11-26,False,False,1,3
2022-11-27,True,True,2,1
2022-11-28,True,False,2,2
2022-11-29,True,False,2,3
2022-11-30,True,False,2,4
2022-12-01,True,False,2,5
2022-12-02,True,False,2,6


In [30]:
non_tracked_days = streak_df.loc[streak_df['tracked']==False,:]
tracked_days = streak_df.loc[streak_df['tracked']==True,:]

In [31]:
non_tracked_days['streak_counter'].max()

3

In [33]:
tracked_days['streak_counter'].max()

7